# Remove nom_9/ Label+Ordinal+Onehot + Logistic auc=0.8033

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import roc_auc_score
from mlxtend.classifier import StackingClassifier


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
 
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = '12-k8EtURQjhX50YB8apJ6yCdgQHXlJ82'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('train.csv')

In [0]:
file2_id = '1mR9Bc0w46nYPs_7Y7LqkehUoZ5_rjz-7'  #雲端硬碟檔案連結碼
downloaded = drive.CreateFile({'id': file2_id})
downloaded.GetContentFile('test.csv')

In [0]:
#from google.colab import files
#uploaded = files.upload()

In [0]:
# Preprocessing

In [0]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
target = train.iloc[:,-1]
#train.drop('target', axis=1, inplace=True)

In [0]:
data = pd.concat([train, test])
data.index = range(data.shape[0])
data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,day,month,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,target
0,0,0,0,T,Y,2,2,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,0.0
1,0,1,0,T,Y,7,8,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,0.0
2,0,0,0,F,Y,7,2,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,0.0
3,0,1,0,F,Y,2,1,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,1.0
4,0,0,0,F,N,7,8,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,0.0


In [0]:
data.columns

Index(['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'day', 'month', 'nom_0',
       'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8',
       'nom_9', 'ord_0', 'ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5',
       'target'],
      dtype='object')

In [0]:
a=[]
b=[]
for i in data['ord_5']:
  a.append(i[0])
  b.append(i[1])
data['ord_5_1'] = a
data['ord_5_2'] = b
data.drop('ord_5', axis=1, inplace=True)

In [0]:
np.unique(b)

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'],
      dtype='<U1')

In [0]:
bin_ = data[['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'ord_0', 'ord_3', 'ord_4', 'nom_9', 'ord_5_2']]
nom_ = data[['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8']]
ord_ = data[['ord_1', 'ord_2', 'day', 'month', 'ord_5_1']]

In [0]:
bin_encode = pd.DataFrame(OrdinalEncoder().fit_transform(bin_))

In [0]:
#hash encoding
bin_encode[8] = bin_encode[8].map(lambda x:hash(x) % 10)

In [0]:
ord_encode = pd.DataFrame()
for i in ord_.columns:
    index = data[:train.shape[0]].groupby(by=i)['target'].mean().sort_values().index
    dict_ = dict(zip(index, range(len(index))))
    ord_encode[i] = ord_[i].map(lambda x : dict_[x])

In [0]:
final_data = pd.concat([bin_encode, ord_encode, nom_], axis=1)

In [0]:
t = time.time()
OneHot = pd.get_dummies(final_data, 
                        columns=nom_.columns, 
                        drop_first=True, 
                        sparse=True
                       )
onhot_time = time.time()-t
onhot_time

4.09924840927124

In [0]:
Xtrain = OneHot.iloc[:train.shape[0]]
Xtest = OneHot.iloc[train.shape[0]:]

In [0]:
#Xtrain = OneHot.iloc[:train.shape[0]].to_coo().tocsr()
#Xtest = OneHot.iloc[train.shape[0]:].to_coo().tocsr()

In [0]:
t = time.time()
LGR = LogisticRegression(random_state=1, C=0.12).fit(Xtrain, target)
LGR_time = time.time()-t
LGR_time

134.35044384002686

In [0]:
LGR_y_prob = LGR.predict_proba(Xtest)

In [0]:
submission_new = pd.DataFrame(zip(test.index,LGR_y_prob[:,1]), columns=['id', 'target'])

In [0]:
submission_new.to_csv('submission_c0.12.csv', index=False)

In [0]:
from google.colab import files
files.download("submission_c0.12.csv")

In [0]:
submission_new

,id,target
0,300000,0.415049
1,300001,0.668705
2,300002,0.128841
3,300003,0.501323
4,300004,0.878833
5,300005,0.126438
6,300006,0.321522
7,300007,0.186919
8,300008,0.554795
9,300009,0.382717


------------------